
Created on Fri Jun 4 15:50 2020 (Author: Clara Burgard)

This is an example script to convert the 2D fields (stereographic grid) of 
- the conservative temperature to potential temperature 
- the absolute salinity to practical salinity

In [ ]:
import xarray as xr
import numpy as np
from tqdm.notebook import tqdm
import gsw
import matplotlib.pyplot as plt
import multimelt.useful_functions as uf
import multimelt.T_S_profile_functions as tspf
import multimelt.melt_functions as meltf

import distributed
import glob

DEPENDING ON THE SIZE OF YOUR DATA, THIS WILL REQUIRE USING DASK

For example, open a client like this: (check that the number of workers is lower or equal to the number of cores you use and the memory limit is equal to the memory of your cores)

In [ ]:
client = distributed.Client(n_workers=4, dashboard_address=':8795', local_directory='/tmp', memory_limit='6GB')

In [ ]:
client # check what's happening with the client

READ IN DATA

In [ ]:
inputpath_data = # path to folder where you stored the file that defines the depth of your depth coordinate and your files with 2D fields of temperature and salinity
inputpath_isf = # path to folder where you stored the masks created earlier
outputpath_profiles = # path to folder where you want to store your output

# make the domain a little smaller to make the computation even more efficient
map_lim = [-3000000,3000000]

In [ ]:
# read file that defines the depth of your depth coordinate
file_mask = xr.open_mfdataset(inputpath_data+'mask_depth_coord_Ant_stereo.nc') 
# read mask file created earlier
file_isf = xr.open_mfdataset(inputpath_isf+'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc', chunks={'x': 533, 'y': 533}) 
# make domain a little smaller
file_isf_cut = uf.cut_domain_stereo(file_isf, map_lim, map_lim)

# 2D field of temperature and salinity for just one time step (to define domains where there is open ocean)
file_TS_orig  = xr.open_mfdataset(inputpath_data+'variables_of_interest_2000_Ant_stereo.nc', chunks={'x': 600, 'y': 600})
# make domain a little smaller
file_TS_orig_cut = uf.cut_domain_stereo(file_TS_orig, map_lim, map_lim).squeeze().drop('time')

In [ ]:
# longitude and latitude
lon = file_isf_cut['longitude']
lat = file_isf_cut['latitude']

FORMAT DATA

Prepare temperature and salinity fields

In [ ]:
# read in all 2D temperature and salinity files (all years in folder) and concatenate them
ts_files = list(sorted(glob.glob(inputpath_data+'variables_of_interest_*_Ant_stereo.nc')))
ds_ts  = xr.open_mfdataset(ts_files, concat_dim='new_time', combine='nested', chunks={'x': 600, 'y': 600})
ds_ts = ds_ts[['votemper', 'vosaline', 'sosst']] # keep conservative temperature, absolute salinity and sea surface temperature

# define the years on the time axis (this is necessary if the yearly separate files all have year "1" as coordinate
yy_start = 1989
yy_end = 2018
ds_ts = ds_ts.squeeze('time') 
ds_ts = ds_ts.rename({'new_time': 'time'})
ds_ts = ds_ts.assign_coords(time=np.arange(yy_start, yy_end+1)) 

# make domain a little smaller
ds_ts_cut = uf.cut_domain_stereo(ds_ts, map_lim, map_lim)

Prepare the depth axis

In [ ]:
nemo_depth = np.round(file_mask['gdept_0'].squeeze(dim=['lon','lat']), 3) # round to mm scale - should be enough

Cut out the temperature and salinity and assign the new depth axis

In [ ]:
ds_temp_saline_input = ds_ts_cut[['votemper', 'vosaline']]
ds_temp_saline_input = ds_temp_saline_input.rename({'votemper': 'temperature', 'vosaline': 'salinity'})
ds_temp_saline_input = ds_temp_saline_input.rename({'deptht': 'depth'})
ds_temp_saline_input['depth'] = np.round(ds_temp_saline_input.depth, 3)
ds_temp_saline_input = ds_temp_saline_input.assign_coords(depth=nemo_depth.values)

In [ ]:
# only points of open ocean
# where there is an SST, it is ocean without ice shelf cavity
mask_ocean = np.isfinite(file_TS_orig_cut['sosst']).squeeze()  

CONVERT CONSERVATIVE TEMPERATURE TO POTENTIAL TEMPERATURE AND ABSOLUTE SALINITY TO PRACTICAL SALINITY

In [ ]:
ds_temp_saline_input['theta_ocean'] = xr.apply_ufunc(gsw.pt_from_CT, ds_temp_saline_input['salinity'].where(mask_ocean), ds_temp_saline_input['temperature'].where(mask_ocean), dask = 'allowed')
ds_temp_saline_input['salinity_ocean'] = xr.apply_ufunc(gsw.SP_from_SA, ds_temp_saline_input['salinity'].where(mask_ocean), ds_temp_saline_input['depth'], lon, lat, dask = 'allowed')
ds_temp_saline_output = ds_temp_saline_input[['theta_ocean', 'salinity_ocean']]

Prepare to write the results to multiple files (1 per year) - this is where dask comes into play

In [ ]:
yearly_datasets = list(tspf.split_by_chunks(ds_temp_saline_output.unify_chunks(),'time'))
paths = [tspf.create_filepath(ds, 'T_S_theta_ocean_corrected', inputpath_profiles, ds.time[0].values) for ds in yearly_datasets]

This takes approximately 1 min per year

In [ ]:
xr.save_mfdataset(datasets=yearly_datasets, paths=paths)